In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



c:\Users\priya\OneDrive\Desktop\German_Credit_Multi_Objective\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [3]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient
#adaptive weights based on distance

results = []
count_alpha = 100
trials_per_alpha = 10
for i in range(count_alpha+1):
    alpha = i/count_alpha
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })

acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 11:10:21,667] A new study created in memory with name: no-name-86e875c0-a5c6-4471-9d0a-526118ea8f01
[I 2025-05-16 11:10:22,046] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.026254334030813502, 'subsample': 0.7195394620484923, 'colsample_bytree': 0.6581935173790406}.
[I 2025-05-16 11:10:22,116] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.07066179061078347, 'subsample': 0.7521668970821306, 'colsample_bytree': 0.993680904362854}.
[I 2025-05-16 11:10:22,193] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 5, 'eta': 0.01179558371066809, 'subsample': 0.7513472931895306, 'colsample_bytree': 0.9469945747362926}.
[I 2025-05-16 11:10:22,269] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.2611482024532094, 'subsample': 0.7471789470029445, 'colsample_bytree': 0.8731524996570348}.
[I 2025-05-

Completed study for alpha=0.0


[I 2025-05-16 11:10:23,019] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.13539512907416912, 'subsample': 0.6467310986130091, 'colsample_bytree': 0.6111933413459553}.
[I 2025-05-16 11:10:23,103] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.21426387012325546, 'subsample': 0.7121491077544685, 'colsample_bytree': 0.7973895174774241}.
[I 2025-05-16 11:10:23,183] Trial 4 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.2643898739844831, 'subsample': 0.8961953694818072, 'colsample_bytree': 0.7093394341047086}.
[I 2025-05-16 11:10:23,262] Trial 5 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 5, 'eta': 0.20856349487726047, 'subsample': 0.9798664015766994, 'colsample_bytree': 0.9094572635087834}.
[I 2025-05-16 11:10:23,334] Trial 6 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0

Completed study for alpha=0.01


[I 2025-05-16 11:10:23,828] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.049834050525281816, 'subsample': 0.6539819227641096, 'colsample_bytree': 0.7317492424216627}.
[I 2025-05-16 11:10:23,914] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.21214919334663507, 'subsample': 0.7954798766229876, 'colsample_bytree': 0.7080082586106248}.
[I 2025-05-16 11:10:23,993] Trial 3 finished with values: [0.6650139538588317, 0.01993528448914837] and parameters: {'max_depth': 7, 'eta': 0.2747902259216701, 'subsample': 0.7565936386487201, 'colsample_bytree': 0.6869956121496366}.
[I 2025-05-16 11:10:24,076] Trial 4 finished with values: [0.5764812381101175, 0.032371566798053246] and parameters: {'max_depth': 6, 'eta': 0.27735882689991403, 'subsample': 0.6612966288669655, 'colsample_bytree': 0.7486740381921451}.
[I 2025-05-16 11:10:24,165] Trial 5 finished with values: [0.5429245817671

...

Completed study for alpha=0.98


[I 2025-05-16 11:12:57,752] Trial 1 finished with values: [0.6864211032585553, 0.0665891335054597] and parameters: {'max_depth': 5, 'eta': 0.16369325871214072, 'subsample': 0.7844342194216521, 'colsample_bytree': 0.770737615816639}.
[I 2025-05-16 11:12:57,916] Trial 2 finished with values: [0.6757190602665784, 0.04034895716408752] and parameters: {'max_depth': 3, 'eta': 0.196395786868484, 'subsample': 0.7416444122277482, 'colsample_bytree': 0.9009127725743217}.
[I 2025-05-16 11:12:58,084] Trial 3 finished with values: [0.6728593616454219, 0.05749723659627575] and parameters: {'max_depth': 3, 'eta': 0.2550349788311222, 'subsample': 0.6372088518505069, 'colsample_bytree': 0.9214831303292089}.
[I 2025-05-16 11:12:58,272] Trial 4 finished with values: [0.6728532348138822, 0.0609571771701263] and parameters: {'max_depth': 5, 'eta': 0.03375874102048622, 'subsample': 0.9509709832300431, 'colsample_bytree': 0.6567636622056527}.
[I 2025-05-16 11:12:58,397] Trial 5 finished with values: [0.66643

Completed study for alpha=0.99


[I 2025-05-16 11:12:59,423] Trial 1 finished with values: [0.6799986520970612, 0.054576348485083] and parameters: {'max_depth': 4, 'eta': 0.14307058672453127, 'subsample': 0.8264001170554022, 'colsample_bytree': 0.9221897283277382}.
[I 2025-05-16 11:12:59,656] Trial 2 finished with values: [0.7192838959296394, 0.08402873400738671] and parameters: {'max_depth': 7, 'eta': 0.27807649487520747, 'subsample': 0.8001316217463867, 'colsample_bytree': 0.7949910763955618}.
[I 2025-05-16 11:12:59,828] Trial 3 finished with values: [0.685002741757114, 0.07168388762564062] and parameters: {'max_depth': 7, 'eta': 0.04036332538497878, 'subsample': 0.8664035899912863, 'colsample_bytree': 0.7214047967526018}.
[I 2025-05-16 11:12:59,965] Trial 4 finished with values: [0.6978614294510667, 0.11725985414206454] and parameters: {'max_depth': 6, 'eta': 0.16730750135215847, 'subsample': 0.7380220611794988, 'colsample_bytree': 0.8269697360278406}.
[I 2025-05-16 11:13:00,165] Trial 5 finished with values: [0.68

Completed study for alpha=1.0
Pareto-optimal solutions:
Params: {'alpha': 0.0, 'max_depth': 7, 'eta': 0.026254334030813502, 'subsample': 0.7195394620484923, 'colsample_bytree': 0.6581935173790406}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01, 'max_depth': 5, 'eta': 0.1307778537955426, 'subsample': 0.8981798914612369, 'colsample_bytree': 0.8371454048093636}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.02, 'max_depth': 7, 'eta': 0.049834050525281816, 'subsample': 0.6539819227641096, 'colsample_bytree': 0.7317492424216627}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.03, 'max_depth': 5, 'eta': 0.2447939514013812, 'subsample': 0.6115316516230084, 'colsample_bytree': 0.7217525130841902}, Accuracy: 0.6729, DPD: 0.0086
Params: {'alpha': 0.14, 'max_depth': 7, 'eta': 0.2182752308817038, 'subsample': 0.9802033726552732, 'colsample_bytree': 0.8723362608110063}, Accuracy: 0.6743, DPD: 0.0144
Params: {'alpha': 0.16, 'max_depth': 5, 'eta': 0.25925623794180147, 'subsample': 0.791

In [4]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [7]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
trials_per_alpha = 50
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=0)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(alpha,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=1)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(alpha,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])
count_alpha = 20



for i in range(count_alpha):
    print(i)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc)-normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd)-normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = np.sqrt(acc_dist + dpd_dist)
        if dist > max_dist:
            max_dist = dist
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 11:13:19,261] A new study created in memory with name: no-name-9f3b9129-059c-4f8e-a533-5ccb0b9fddc4
[I 2025-05-16 11:13:19,357] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}.
[I 2025-05-16 11:13:19,460] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.04040577032334644, 'subsample': 0.7488904515273311, 'colsample_bytree': 0.9586222733019236}.
[I 2025-05-16 11:13:19,567] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.14583696499170543, 'subsample': 0.795810321715773, 'colsample_bytree': 0.952224552097848}.
[I 2025-05-16 11:13:19,666] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.13623371217287475, 'subsample': 0.6081598975377428, 'colsample_bytree': 0.7401898808501025}.
[I 2025-05-1

Completed study for alpha=0


[I 2025-05-16 11:13:25,117] Trial 1 finished with values: [0.7021379578657795, 0.06314616230362322] and parameters: {'max_depth': 6, 'eta': 0.26589168923854556, 'subsample': 0.7897562282613623, 'colsample_bytree': 0.9600111848863682}.
[I 2025-05-16 11:13:25,260] Trial 2 finished with values: [0.6764374312646085, 0.045053618695635365] and parameters: {'max_depth': 4, 'eta': 0.06306784136966327, 'subsample': 0.74798241185463, 'colsample_bytree': 0.6551612112285078}.
[I 2025-05-16 11:13:25,425] Trial 3 finished with values: [0.6978491757879871, 0.08431029941171075] and parameters: {'max_depth': 6, 'eta': 0.18242492595888532, 'subsample': 0.968494180092288, 'colsample_bytree': 0.9277864888887004}.
[I 2025-05-16 11:13:25,594] Trial 4 finished with values: [0.692148159040293, 0.04711257309011945] and parameters: {'max_depth': 6, 'eta': 0.15633414111361582, 'subsample': 0.7677602996238255, 'colsample_bytree': 0.6515532718065272}.
[I 2025-05-16 11:13:25,738] Trial 5 finished with values: [0.69

Completed study for alpha=1
[{'accuracy': 0.5000061268315397, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}}, {'accuracy': 0.7214129698896864, 'dpd': 0.10581076448863501, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 7, 'eta': 0.27926006398945585, 'subsample': 0.8218636333494963, 'colsample_bytree': 0.8238601358476214}}]
0


[I 2025-05-16 11:13:32,507] Trial 1 finished with values: [0.6742945719335974, 0.029715368588148706] and parameters: {'max_depth': 3, 'eta': 0.2624127443857211, 'subsample': 0.6398351480254919, 'colsample_bytree': 0.9273198177738456}.
[I 2025-05-16 11:13:32,593] Trial 2 finished with values: [0.6742976353493672, 0.057803156772305846] and parameters: {'max_depth': 3, 'eta': 0.28080783237841106, 'subsample': 0.6181718122808142, 'colsample_bytree': 0.837916413058396}.
[I 2025-05-16 11:13:32,701] Trial 3 finished with values: [0.666432315360273, 0.04994182214162388] and parameters: {'max_depth': 4, 'eta': 0.1705686695540982, 'subsample': 0.9161626847024184, 'colsample_bytree': 0.8485905178665168}.
[I 2025-05-16 11:13:32,839] Trial 4 finished with values: [0.6828660092576424, 0.045312916176632] and parameters: {'max_depth': 7, 'eta': 0.1203245017352115, 'subsample': 0.794519582525799, 'colsample_bytree': 0.6548483030696385}.
[I 2025-05-16 11:13:32,944] Trial 5 finished with values: [0.67500

...

Completed study for alpha=0.02252197265625
18


[I 2025-05-16 11:15:29,313] Trial 1 finished with values: [0.6657261980253222, 0.016923952882237265] and parameters: {'max_depth': 7, 'eta': 0.2962557453904555, 'subsample': 0.8505436791396601, 'colsample_bytree': 0.9185126430559836}.
[I 2025-05-16 11:15:29,423] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 4, 'eta': 0.05675002143412593, 'subsample': 0.8794169283264468, 'colsample_bytree': 0.851588780909592}.
[I 2025-05-16 11:15:29,543] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 7, 'eta': 0.05391397292806195, 'subsample': 0.9397143497197321, 'colsample_bytree': 0.660788860795329}.
[I 2025-05-16 11:15:29,653] Trial 4 finished with values: [0.6692966091050844, 0.022221463950426807] and parameters: {'max_depth': 6, 'eta': 0.28499925848496943, 'subsample': 0.6676799485288464, 'colsample_bytree': 0.6757276737239967}.
[I 2025-05-16 11:15:29,754] Trial 5 finished with values: [

Completed study for alpha=0.022491455078125
19


[I 2025-05-16 11:15:35,284] Trial 1 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 4, 'eta': 0.23403115822266618, 'subsample': 0.7981434372239434, 'colsample_bytree': 0.8778856782504978}.
[I 2025-05-16 11:15:35,405] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 3, 'eta': 0.15653418706676264, 'subsample': 0.7115916132762418, 'colsample_bytree': 0.8443908848876551}.
[I 2025-05-16 11:15:35,544] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.03432410931468004, 'subsample': 0.9225001339521833, 'colsample_bytree': 0.923817762955518}.
[I 2025-05-16 11:15:35,646] Trial 4 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.16109160570667802, 'subsample': 0.7853296256998328, 'colsample_bytree': 0.7854917304418363}.
[I 2025-05-16 11:15:35,754] Trial 5 finished with values:

Completed study for alpha=0.0224761962890625
Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.015625, 'max_depth': 5, 'eta': 0.05282487573863137, 'subsample': 0.8688659694886003, 'colsample_bytree': 0.6687411708124102}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01953125, 'max_depth': 5, 'eta': 0.0504980502577411, 'subsample': 0.6485748884994879, 'colsample_bytree': 0.6209503294456052}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021484375, 'max_depth': 3, 'eta': 0.056966858952711155, 'subsample': 0.6032123055973961, 'colsample_bytree': 0.7679043893131532}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.02197265625, 'max_depth': 5, 'eta': 0.19607347343993498, 'subsample': 0.6032555369919693, 'colsample_bytree': 0.734748734600021}, Accuracy: 0.5522, DPD: 0.0037
Params: {'alpha': 0.0224609375, 'max_depth': 3,

In [8]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()